# BUILDING OUR DATAFRAME -- notebook 1

This notebook serves to build our dataframe using sential 2 data.

Author: Xuyuan Zhang; Date: Feb. 15. 2024

In [1]:
# necessary packages in this notebook
import os
import glob
import zipfile
import pandas as pd
import numpy as np 
import geopandas as gpd
import rasterio

## We first download the data from the official website

After downloading all of the files necessary to our analysis, we unzip all of the codes to the directory. This code is for unzip all of the data, and to execute it, you need to make sure you have enough space in your computer.

In [2]:
pwd()

'e:\\umich\\final project\\notebook'

In [3]:
os.chdir('../data')

In [4]:
# give a fresh new start by deleting all of the files that already exists in the folder

original_zip_files = ['S2B_MSIL2A_20230608T161829_N0509_R040_T16TGN_20230608T192144.SAFE.zip',
                      'S2B_MSIL2A_20230608T161829_N0509_R040_T17TKG_20230608T192144.SAFE.zip',
                      'S2B_MSIL2A_20230618T161829_N0509_R040_T17TLG_20230618T191812.SAFE.zip',
                      'S2B_MSIL2A_20230618T161829_N0509_R040_T17TLH_20230618T191812.SAFE.zip']

def delete_files_except(directory, keep_filenames):
    # Ensure that the path is absolute and normalized
    directory = os.path.abspath(directory)

    # Convert keep_filenames to a set for faster membership testing
    keep_filenames_set = set(keep_filenames)

    # Check if it's a directory, just in case
    if not os.path.isdir(directory):
        print(f"The directory {directory} does not exist or is not a directory.")
        return

    # Walk through the directory
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Construct the full path of the file
            file_path = os.path.join(root, file)
            # Check if the file is not in the list of filenames to keep
            if file not in keep_filenames_set:
                try:
                    os.remove(file_path)
                    print(f"Deleted: {file_path}")
                except OSError as e:
                    print(f"Error: {e.strerror}, while trying to delete file {file_path}")

delete_files_except('', original_zip_files)

#unzip fl.zip using zipfile
for file in original_zip_files:
    with zipfile.ZipFile(file, 'r') as zip_ref:
        zip_ref.extractall()

Deleted: e:\umich\final project\data\mosaic.tfw
Deleted: e:\umich\final project\data\mosaic.tif
Deleted: e:\umich\final project\data\mosaic.tif.aux.xml
Deleted: e:\umich\final project\data\mosaic.tif.ovr
Deleted: e:\umich\final project\data\mosaic.tif.xml
Deleted: e:\umich\final project\data\mosaic_clipped.tfw
Error: 另一个程序正在使用此文件，进程无法访问。, while trying to delete file e:\umich\final project\data\mosaic_clipped.tif
Error: 另一个程序正在使用此文件，进程无法访问。, while trying to delete file e:\umich\final project\data\mosaic_clipped.tif.ovr
Deleted: e:\umich\final project\data\mosaic_clipped.tif.xml
Deleted: e:\umich\final project\data\S2B_MSIL2A_20230608T161829_N0509_R040_T16TGN_20230608T192144.SAFE.tiff
Deleted: e:\umich\final project\data\S2B_MSIL2A_20230608T161829_N0509_R040_T17TKG_20230608T192144.SAFE.tiff
Deleted: e:\umich\final project\data\S2B_MSIL2A_20230618T161829_N0509_R040_T17TLG_20230618T191812.SAFE.tiff
Deleted: e:\umich\final project\data\S2B_MSIL2A_20230618T161829_N0509_R040_T17TLH_20230618T1

## Clean the data and combine to the final tiff file

Our dataset is ready with all 13 bands of information, though we just need red, blue, and green bands to create an RGB image. We will write the data of these three bands in an empty tiff file and mask our region of interest on it. Locate the folder named ‘R10’ and change the names of files in the code below accordingly.

In [5]:
folder_name = [filename.replace('.zip', '') for filename in original_zip_files]

I decide to write a code to automatically extract the given folder's data in the directory and then combine the results to one

In [8]:
# Pattern to match any folder under GRANULE that starts with 'L2A_'
for base_dir in folder_name:
    granule_pattern = f"{base_dir}\GRANULE/L2A_*"
    granule_folders = glob.glob(granule_pattern)
    
    # Process each GRANULE folder found
    for granule_folder in granule_folders:
        # print(granule_folder)
        
        blue_band_path = glob.glob(os.path.join(granule_folder, 'IMG_DATA', 'R10m', '*_B02_10m.jp2'))[0]
        green_band_files = glob.glob(os.path.join(granule_folder, 'IMG_DATA', 'R10m', '*_B03_10m.jp2'))[0]
        red_band_path = glob.glob(os.path.join(granule_folder, 'IMG_DATA', 'R10m', '*_B04_10m.jp2'))[0]
        nir_band_path = glob.glob(os.path.join(granule_folder, 'IMG_DATA', 'R10m', '*_B08_10m.jp2'))[0]
        # to construct the full path to each band file
        blue = rasterio.open(blue_band_path)
        green = rasterio.open(green_band_files) 
        red = rasterio.open(red_band_path) 
        nir = rasterio.open(nir_band_path)
        print('finished reading file')
        
        with rasterio.open('{}.tiff'.format(base_dir),'w',driver='Gtiff', width=blue.width, height=blue.height, count=4, crs=blue.crs,transform=blue.transform, dtype=blue.dtypes[0]) as rgb_nir:
            rgb_nir.write(blue.read(1), 1)
            print('Write the blue band')
            rgb_nir.write(green.read(1), 2)
            print('Write the green band')
            rgb_nir.write(red.read(1), 3)
            print('Write the red band')
            rgb_nir.write(nir.read(1), 4)
            print('Write the NIR band')
            rgb_nir.close()
            print('finished')

finished reading file
Write the blue band
Write the green band
Write the red band
Write the NIR band
finished
finished reading file
Write the blue band
Write the green band
Write the red band
Write the NIR band
finished
finished reading file
Write the blue band
Write the green band
Write the red band
Write the NIR band
finished
finished reading file
Write the blue band
Write the green band
Write the red band
Write the NIR band
finished


Sometimes, our data may have multiple coordinate system, which will make the merging fail. Therefore, we have to reproject the data so that we can have a uniform coordinate system. The reprojection and other analysis will use arcpy and we should use the other notebook to do that. Hopefully, in this result, we are using the same WGS 84! Therefore, we would not need to revise it! Below code is for check, and can revise the code using a for loop and extract the unique value by constructing a list and taking set.

In [9]:
intput_tiff_name = [filename.replace('.zip', '.tiff') for filename in original_zip_files]

In [10]:
with rasterio.open(intput_tiff_name[2]) as src:
    dtype = src.dtypes[0]  # Data type of the first band
    num_bands = src.count  # Number of bands
    coordinate_system = src.crs  # Coordinate reference system

    print(f"Data Type: {dtype}")
    print(f"Number of Bands: {num_bands}")
    print(f"Coordinate System: {coordinate_system}")

Data Type: uint16
Number of Bands: 4
Coordinate System: EPSG:32617


In [11]:
with rasterio.open(intput_tiff_name[0]) as src:
    dtype = src.dtypes[0]  # Data type of the first band
    num_bands = src.count  # Number of bands
    coordinate_system = src.crs  # Coordinate reference system

    print(f"Data Type: {dtype}")
    print(f"Number of Bands: {num_bands}")
    print(f"Coordinate System: {coordinate_system}")

Data Type: uint16
Number of Bands: 4
Coordinate System: EPSG:32616
